In [1]:
import ROOT
import math
from ROOT import TH1F, TF1, TColor
from scipy.stats import norm

rndmGenerator = ROOT.TRandom3()

def bhathat(mu, s, tau, n, m):
    return (n+m-(1+tau)*mu*s)/(2+2*tau)+(((n+m-(1+tau)*mu*s)**2+(1+tau)*m*mu*s)/(4*(1+tau)**2))**0.5
def factorial(n):
    n_factorial = 1
    if n < 0:
        print("error")
    elif n == 0:
        n_factorial = 1
    else: 
        for i in range(1,n + 1):
            n_factorial = n_factorial*i
    return n_factorial

def likelihood(mu, s, b, tau, n, m):
    return (mu*s+b)**n*math.exp(-(mu*s+b))*(tau*b)**m*math.exp(-tau*b)/(factorial(n)*factorial(m))

def qmu(mu, s, b, tau):
    n=rndmGenerator.Poisson(mu*s+b)
    m=rndmGenerator.Poisson(tau*b)
    mu_hat = (n-m/tau)/s
    b_hat = m/tau
    b_hat_hat = bhathat(mu, s, tau, n, m)
    if mu_hat < mu:
        likelihood_mu_bhat = likelihood(mu, s, b_hat_hat, tau, n, m)
        likelihood_muhat_bhathat = likelihood(mu_hat, s, b_hat, tau, n, m)
        likelihoodRatio_mu = likelihood_mu_bhat/likelihood_muhat_bhathat
        q_mu = -2*math.log(likelihoodRatio_mu)
    else:
        q_mu = 0
    return q_mu

def q0(mu, s, b, tau, n, m):
#     n=rndmGenerator.Poisson(mu*s+b)
#     m=rndmGenerator.Poisson(tau*b)
    mu_hat = (n-m/tau)/s
    b_hat = m/tau
    b_hat_hat = bhathat(mu, s, tau, n, m)
    if mu_hat >= 0:
        likelihood_mu_bhat = likelihood(mu, s, b_hat_hat, tau, n, m)
        likelihood_muhat_bhathat = likelihood(mu_hat, s, b_hat, tau, n, m)
        likelihoodRatio_mu = likelihood_mu_bhat/likelihood_muhat_bhathat
        q_mu = -2*math.log(likelihoodRatio_mu)
    else:
        q_mu = 0
    return q_mu

# print ("2:", factorial(4) )
# print ("e : ", log_likelihood(0,2,2,1,2,2) )

Welcome to JupyROOT 6.20/06


In [2]:
%jsroot on

In [3]:
def func_fq0(q0):
    if q0[0] > 1.e-10:
        fq = 0.5*(2*math.pi)**(-0.5)*(q0[0])**(-0.5)*math.exp(-0.5*q0[0])
    else:
        fq = 1
    return fq

fq0 = TF1("fq0",func_fq0,0,40)

sigma_A = (1.*1. / q0(0,10,10,1,20,10))**0.5
# mu_ = 1
print(q0(0,10,10,1,20,10))
def func_fq0_mu(q_0):
    sigma_A = (0.29429242768582003)**0.5
    if q_0[0] > 0.1:
        fq = 0.5*(2*math.pi)**(-0.5)*(q_0[0])**(-0.5)*math.exp(-0.5*(q_0[0]**0.5 - 1/sigma_A)**2)
    else:
        q = 0.1
        fq = 1-norm.cdf(1/sigma_A) + 0.5*(2*math.pi)**(-0.5)*(q)**(-0.5)*math.exp(-0.5*(q**0.5 - 1/sigma_A)**2)
#         fq = 1-norm.cdf(1/sigma_A)
    return fq
fq0_mu = TF1("fq0_mu",func_fq0_mu,0,40)

# c_f = ROOT.TCanvas()
# c_f.SetLogy()
# fq0.Draw()
# fq0_mu.Draw()
# c_f.Draw()

3.39798073590795


In [4]:
h_mu0 = ROOT.TH1F("h_mu0","f(q0|0);q_{0};f(q_{0}|0)",20, 0, 40)
h_mu1 = ROOT.TH1F("h_mu1","f(q0|1);q_{0};f(q_{0}|1)",20, 0, 40)

mu=0
s=10
b=10
tau=1
for i in range(10000000):
    h_mu0.Fill(q0(0, s, b, tau, rndmGenerator.Poisson(0*s+b), rndmGenerator.Poisson(tau*b)))
    h_mu1.Fill(q0(0, s, b, tau, rndmGenerator.Poisson(1*s+b), rndmGenerator.Poisson(tau*b)))

In [6]:
c = ROOT.TCanvas()
c.SetLogy()
print(h_mu1.Integral())
h_mu0.Scale(0.5/h_mu0.Integral())
h_mu1.Scale(0.5/h_mu1.Integral())
h_mu0.SetMaximum(10)
h_mu0.SetMinimum(10e-9)
h_mu0.SetLineColor(2) # red
h_mu0.SetLineWidth(2)
h_mu1.SetLineColor(4) # blue
h_mu1.SetLineWidth(3)
h_mu1.SetLineStyle(3)
h_mu1.Draw("hist")
h_mu0.Draw("hist same")
fq0.SetLineColor(1)
fq0.Draw("same")
fq0_mu.SetLineColor(1)
fq0_mu.Draw("same")
c.Draw()

1.000000002878096
